In [33]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

from typing import Dict, Text
import numpy as np

In [42]:
ratings = tfds.load('movielens/100k-ratings', split='train')
movies = tfds.load('movielens/100k-movies', split='train')

ratings = ratings.map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})

# movies = movies.map(lambda x: {
#     'movie_title': x['movie_title']
# })

In [43]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

movie_titles = movies.batch(1000).map(lambda x: x['movie_title'])
user_ids = ratings.batch(1000000).map(lambda x: x['user_id'])

uniq_movie_titles = np.unique(np.concatenate(list(movie_titles)))
uniq_user_ids = np.unique(np.concatenate(list(user_ids)))

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 0. First element had shape [1] and element 1 had shape [2]. [Op:IteratorGetNext]

In [40]:
embedding_dimension = 32
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=uniq_user_ids),
    tf.keras.layers.Embedding(len(uniq_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=uniq_user_ids),
    tf.keras.layers.Embedding(len(uniq_movie_titles) + 1, embedding_dimension)
])

metrics = tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128).map(movie_model),
    k=100
)

task = tfrs.tasks.Retrieval(
    metrics=metrics
)

TypeError: Exception encountered when calling layer 'string_lookup_15' (type StringLookup).

Expected string, but got Tensor("args_0:0", shape=(None,), dtype=string) of type 'Tensor'.

Call arguments received by layer 'string_lookup_15' (type StringLookup):
  • inputs={'movie_title': 'tf.Tensor(shape=(None,), dtype=string)'}

In [37]:
class MovieLensModel(tfrs.Model):
    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features['user_id'])
        positive_movie_embeddings = self.movie_model(features['movie_title'])
        
        return self.task(user_embeddings, positive_movie_embeddings)

In [38]:
model = MovieLensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=2)

Epoch 1/2


TypeError: in user code:

    File "d:\Work\movie-recommendation-system\.venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "d:\Work\movie-recommendation-system\.venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Work\movie-recommendation-system\.venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "d:\Work\movie-recommendation-system\.venv\lib\site-packages\tensorflow_recommenders\models\base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "C:\Users\nurok\AppData\Local\Temp\ipykernel_12168\2190547765.py", line 12, in compute_loss
        return self.task(user_embeddings, positive_movie_embeddings)
    File "d:\Work\movie-recommendation-system\.venv\lib\site-packages\kera